<a id="inicio"></a>
<img src="./figs/barra_uclm_esiiab.png" alt="Banner UCLM - ESIIAB" align="right">

<br><br><br>
<h1><font color="#B30033" size=5>Intelligent Systems - Course 2021-2022</font></h1>



<h1><font color="#999999" size=5>Assignment 1: Non Informed Search Algorithms</font></h1>


<br><br>
<div style="text-align: left">
<font color="##4E70BE" size=3>Lecturers:</font><br>
<ul>
  <li><font color="#999999" size=3>Guillermo Tomás Fernández Martín</font><br></li>
  <li><font color="#999999" size=3>José Antonio Gámez Martín</font><br></li>
  <li><font color="#999999" size=3>Ismael García Varea</font><br></li>
  <li><font color="#999999" size=3>Luis González Naharro</font><br></li>
</ul>
</div>

<br>

<font color="##4E70BE" size=3>Students:</font><br>
<ul>
  <li><font color="#999999" size=3>Pablo Lario Gómez</font><br></li>
  <li><font color="#999999" size=3>Diego Miguel López García</font><br></li>
</ul>
</div>

<br>

## Introduction
In this assignment we will study and put into practice non informed search algorithms. To do that, some of the algorithms seen in unit 2 will be implemented and used to solve a particular problem: maze pathfinding.
We will also analyze and compare the performance of the algorithms by running them over different instances of the problem.

## Problem description
The maze is a grid of size N x M formed by a set of cells, some of which can be occupied by walls, which cannot be crossed. The rest of the cells will be empty and they will represent the free space where the robot can move horizontally or vertically. In addition, we can have cells with garbage, which must be cleaned by our agent, a vacuum cleaner robot. 

The objective of our robot is to clean the whole area as fast as possible. In other words: **find the shortest path to find all the garbage cells in the environment**. In order to implement our robot we have to take into account that: 
- The robot can start in a random cell of the map.
- The robot can move horizontally or vertically in the maze.
- The robot can not cross walls or go beyond the limits of the maze.
- The robot will have to clean all the garbage cells found in the map, which will be automatically cleaned as soon as the robot arrives to those cells.
- All the movements of the robot will have a cost of 1.
- The search will finish once all garbage cells in the map have been cleaned.

## Provided code

In the following we provide you some of the clases, implemented in `Python` that will help you to develop this assignment. 

First, we will import the necessary classes we need from the Python libraries

In [85]:
import math
import copy
import time

from abc import ABC, abstractmethod


Next, we will import some custom functions from the file `utils.py`. You don't need to modify those functions for the code to work, but feel free to have a look at them if you are curious.

In [86]:
from utils import *

Finally, we will import some third party libraries. We will use those to display the problem in a graphical environment. In order to do that, we will use the magic functions from jupyter to install the library from inside the notebook

In [87]:
!pip install ipythonblocks

In [88]:
from ipythonblocks import BlockGrid
from IPython.display import clear_output

In order to complete the requested search algorithms, we provide you some fundamental classes: 

#### Class `Action`
This class provides the **representation of the actions** that will be performed by the robot. You don't have to modify the code of this class. The possible actions will be: "UP", "DOWN", "RIGHT", "LEFT".

In [89]:
class Action:
    #actions = ["UP", "DOWN", "RIGHT", "LEFT"]

    def __init__(self, move):
        self.move = move

    def __str__(self):
        return f"({self.move})"

    def getCost(self):
        return 1.0

#### Class `State`. 
This class provides the **representation of a state** in the search space. In this problem, a state is defined by the position of the robot and the garbages left in the maze. Note that the maze itself does not need to be part of the state given that it does not change during the search, i.e. walls are fixed during the search. You don't have to modify the code of this class.

The class `State` has an `applyAction` method that, given a valid `Action`, returns a new `State` with the action applied.

In [90]:
class State:

    def __init__(self, pos, garbage):
        self.pos = pos
        self.garbage = garbage

    # equals method. Returns true if the states are the same. 
    # Used for the hash table comparison, compares if both states are equal
    def __eq__(self, state):
        return self.pos == state.pos and self.garbage == state.garbage

    def __str__(self):
        return f"Position: {self.pos}\nGarbage: {self.garbage}"

    def __hash__(self):
        h = 0
        for g in self.garbage:
            h += (math.pow(10,3) * (self.pos[0]+1) + (self.pos[1]+2))
        return int(h)

    def applyAction(self, action):
        st = copy.deepcopy(self)
        
        if (action.move == "UP"):
            st.pos = (st.pos[0]-1,st.pos[1])
        elif (action.move == "DOWN"):
            st.pos = (st.pos[0]+1,st.pos[1])
        elif (action.move == "RIGHT"):
            st.pos = (st.pos[0],st.pos[1]+1)
        elif (action.move == "LEFT"):
            st.pos = (st.pos[0],st.pos[1]-1)
        else:
            print("\n*** ERROR ***: Action " + action + "  is not allowed .....\n")
            sys.exit()
            
        # if the new position has garbage clean it
        if (st.pos in st.garbage):
            st.garbage.remove(st.pos)
        
        return st

#### Class `Node`. 
This class provides a **representation of a node** in the search tree/graph. It contains the state it represents, its parent node and the action taken to reach the current node. You don't have to modify this class. 

The class `Node` also has some methods to provide access to its attributes.

In [91]:
class Node:
    def __init__(self, state, parent, action):
        self.state = state # Must be State Class
        self.parent = parent # Must be Node Class
        self.action = action # Must be Action Class
        self.depth = 0
        self.gCost = 0.0

    def __str__(self):
        return f"depth: {self.depth} {self.state}"

    def __eq__(self, other):
        if not isinstance(other, Node):
        # don't attempt to compare against unrelated types
            return NotImplemented
        return self.state == other.state

    def getState(self):
        return self.state

    def getAction(self):
        return self.action
    
    def getParent(self):
        return self.parent

    def getDepth(self):
        return self.depth
    
    def getGCost(self):
        return self.gCost

#### Class `Problem`
This class provides the **representation of the search problem**. It contains the size of the maze (`rows` and `cols`), the `initialState` and the `maze`. This class can read from a file an instance of the problem to solve, or it can generate a random instance using the size of the grid, the `seed`, the maximum number of divisions (walls) in the maze and the list of garbage cells. You don't have to modify this class. 

The class `Problem` also has some methods to provide access to the initial state, the possible actions and to check if a specific state is the final/goal.

In [92]:
class Problem:

    actions = ["UP", "DOWN", "RIGHT", "LEFT"]

    def __init__(self, rows, cols, seed, maxDivisions, garbageCount, filename=""):

        if (filename != ""):
            self.rows, self.cols, self.maze = readProblemInstance(filename)
            print('Problem read with size',rows,'x',cols)
        else:
            self.rows = rows
            self.cols = cols
            self.maze = getProblemInstance(rows, cols, maxDivisions, garbageCount, seed)

        self.garbage = []

        for r in range(rows):
            for c in range(cols):
                if self.maze[r][c] == 2:
                    init_point = (r,c)
                elif self.maze[r][c] == 3:
                    self.garbage.append((r,c))

        self.initialState = State(init_point, self.garbage)

    def getInitialState(self):
        return self.initialState

    def getActions(self):
        return self.actions

    def isGoal(self, s):
        '''
        check if the given state is final or not
        '''
        return s.garbage == []

## Implementation
In the following we provide you some classes and pieces of code that you will have to complete as a part of this assignment.

#### Class `Search`

The `Search` class is in abstract class that contains some attributes:
- The `problem`to solve.
- The list of `open` nodes, i.e. nodes in the frontier.
- The list of `closed` nodes to implement the graph search, which is implemented using a `set` data structure.
- The attributes to account for the number of generated and expanded nodes, as well as the maximum number size of the closed and open 'lists' to know the maximum number of nodes stored simultaneously in memory. Theses attributes are helpful to estimate the time and memory complexity of the algorithms.

This class also provides three methods:
- `insertNode(self, node, node_list)`: this is an abstract method that has to be implemented by all (search) classes that inherit from `Search`. This method has already been implemented. You don't have to modify it, but you have to implement the correct version in each new search class you create.
- `getSuccesors(self, node)`: this method implements the successors function and should return a list with all the valid successors of a given node. You must program this method.
- `doSearch(self)`: this method implements the graph search you have studied in class. You must program this method. It also provides some statistics of the search process.

In [93]:
class Search(ABC):

    def __init__(self,problem):
        self.problem = problem
        self.open = []
        self.closed = set()

        self.generatedNodes = 0
        self.expandedNodes = 0
        self.exploredMaxSize = 0
        self.openMaxSize = 0

    @abstractmethod
    def insertNode(self, node, node_list):
        pass

    def getSuccesors(self, node):

        self.expandedNodes += 1
        suc = []
        
        for a in self.problem.getActions():
            action = Action(a)
            auxState = node.state.applyAction(action)

            # We check if the state is inside the maze and not out of bounds
            if (auxState.pos[0] < self.problem.rows and 
                auxState.pos[0] >= 0 and
                auxState.pos[1] < self.problem.cols and
                auxState.pos[1] >= 0):
              cell = self.problem.maze[auxState.pos[0]][auxState.pos[1]]
              
              # We check if the new state will not lead to a wall
              if cell != 1:
                nodeAux = Node(node.state.applyAction(action), node, action)
                suc.append(nodeAux)
                self.generatedNodes += 1
        
        return suc

    # Parameter limit is a number that refers to the limit that algorithms depthLimited
    # and IterateDepthLimited use to know until when to stop.
    def doSearch(self, limit):
        totalCost = 0

        self.limit = limit

        current = Node(self.problem.getInitialState(), None, None)

        actionSequence = []

        self.insertNode(current,self.open)
        self.generatedNodes += 1

        finish = False
        
        while (self.open != []):
            # We pick the first node stored in the list
            current = self.open[0] 

            # We remove the node from the open list
            self.open.remove(current) 

            # We check whether the node we have picked from open has been explored
            if current.getState() not in self.closed: 
              # We check if the state of the node is the goal state we want
              if self.problem.isGoal(current.getState()):
                  # If it is a goal node, we finish and recover path
                  finish = True
                  break;
            
              suc = self.getSuccesors(current)
              for s in suc:

                if (s.getParent() != None):
                  s.depth = s.getParent().getDepth() + 1

                self.insertNode(s, self.open)

              self.closed.add(current.getState())
        
        if finish:
          while (current.getParent() != None):
            actionSequence.append(current.getAction())
            current = current.getParent();
        
        return actionSequence, self.generatedNodes, self.expandedNodes
    

#### Class `RandomSearch`
The class `RandomSearch`, inherits from `Search` and implements a random search . Once the `getSuccessors(self,node)` and `doSearch(self)` methods have already been implemented in the parent class, here we only have to implement the `insertNode(self, node, node_list)` method, which in this case inserts the `node`into the `node_list` in a randomly selected position, i.e. not following any search strategy. This code is provided to you in the following cell, and it will allow you to test the implementation of the previous funcions as soon as you implement them.

In [94]:
class RandomSearch(Search):
    def insertNode(self, node, node_list):
        node_list.insert(random.randrange(len(node_list) + 1), node)

In [95]:
def func():
  return 1,[2,3]

In [96]:
a,b = func()
a

1

#### Class `DepthFirst`, `BreadthFirst` and `DepthLimited`

These three classes also inherit from `Search` and will implement the depth first, breadth first and depth limited search strategies.

You have to implement these three classes.

Optionally you might want to implement also the iterative depth limited search strategy (`IterativeDepthLimited`). That will be taken into account to improve your mark of the assigment.

In [97]:
class DepthFirst(Search):
    def insertNode(self, node, node_list):
      node_list.insert(0, node)

In [98]:
class BreadthFirst(Search):
    def insertNode(self, node, node_list):      
      node_list.append(node)

In [99]:
class DepthLimited(Search):
    def insertNode(self, node, node_list):
      if (node.getDepth() <= self.limit):
        node_list.insert(0, node)

In [107]:
class IterateDepthLimited(Search):
    def insertNode(self, node, node_list):
      if (node.getDepth() < self.limit):
        node_list.insert(0, node)

#### The `main` function

Next, we provide you the `main` function that creates the problem and solves it using the search algorithm provided. This method should be used afterwards to carry out the experimentation to study the behaviour of the implemented algorithms for different values of the parameters provided (size of the maze, maximum number of walls, number of garbage cells, and algorithm).

In [101]:
# We have included limit parameter in main so that it is easier for use to test
# the code and generated statistics for DepthLimited and IterateDepthLimited algorithms
def main(limit, rows, cols, seed, maxDivisions, garbageCount, algorithm, configFile=""):
    problem = Problem(rows, cols, seed, maxDivisions, garbageCount, configFile)

    print("$ python assignment1 " + str(problem.rows) + " " + str(problem.cols) + " " + str(seed) + " " + str(maxDivisions) + " " + str(garbageCount) + " " + algorithm + " " + configFile + "\n")
    print("Problem instance:")
    printMaze(problem.maze)
    print("")

    search = None
    i = 0

    if algorithm == "BreadthFirst":
        search = BreadthFirst(problem)
    elif algorithm == "DepthFirst":
        search = DepthFirst(problem)
    elif algorithm == "DepthLimited":
        search = DepthLimited(problem)
        i = limit
    elif algorithm == "RandomSearch":
        search = RandomSearch(problem)
    elif algorithm == "IterateDepthLimited":
        search = IterateDepthLimited(problem)
    else:
        raise Exception

    time_start = time.perf_counter()
    
    tGeneratedNodes = 0
    tExpandedNodes = 0

    path,generatedNodes,expandedNodes = search.doSearch(i)

    if algorithm == "IterateDepthLimited":
      print("---------------------------------------------------")
      print("i = " + str(i) )
      print("Nodos generados: " + str(generatedNodes))
      print("Nodos expandidos: " + str(expandedNodes))

    tGeneratedNodes += generatedNodes
    tExpandedNodes += expandedNodes

    if algorithm == "IterateDepthLimited":
      while(path == []):
        i += 1
        print("---------------------------------------------------")
        print("i = " + str(i) )
        problem = Problem(rows, cols, seed, maxDivisions, garbageCount, configFile)
        search = IterateDepthLimited(problem)
        path,generatedNodes,expandedNodes = search.doSearch(i)
        tGeneratedNodes += generatedNodes
        tExpandedNodes += expandedNodes

        if (path == []):
          print("No se ha encontrado una solución al problema.")

        print("Nodos generados: " + str(generatedNodes))
        print("Nodos expandidos: " + str(expandedNodes))

    time_end = time.perf_counter()

    if algorithm == "IterateDepthLimited":
      print("")

    print("TOTAL VALUES OBTAINED")
    print("Generated nodes: " + str(tGeneratedNodes))
    print("Expanded nodes: " + str(tExpandedNodes))

    print("")
    print("Elapsed time: " + str(time_end - time_start) + " seconds")

    i = len(path) - 1
    newPath = []
    while (i >= 0):
      newPath.append(path[i])
      i -= 1

    for a in newPath:
      print(a)

    return newPath, problem

#### Test your code

Here you have a piece of code to test your implementation. For example, the code to execute a random search technique can be:

In [108]:
# Parameter list

rows = 4
cols =4
seed = 2
maxDivisions = 1
garbageCount = 1
limit = 6
algorithm = "IterateDepthLimited"

path_sol, problem_instance = main(limit, rows, cols, seed, maxDivisions, garbageCount, algorithm)

$ python assignment1 4 4 2 1 1 IterateDepthLimited 

Problem instance:
[3, 0, 0, 0]
[0, 0, 0, 0]
[2, 0, 0, 0]
[0, 0, 0, 0]

---------------------------------------------------
i = 0
Nodos generados: 1
Nodos expandidos: 0
---------------------------------------------------
i = 1
No se ha encontrado una solución al problema.
Nodos generados: 4
Nodos expandidos: 1
---------------------------------------------------
i = 2
No se ha encontrado una solución al problema.
Nodos generados: 13
Nodos expandidos: 4
---------------------------------------------------
i = 3
Nodos generados: 24
Nodos expandidos: 7

TOTAL VALUES OBTAINED
Generated nodes: 42
Expanded nodes: 12

Elapsed time: 0.007540335000157938 seconds
(UP)
(UP)


#### Printing the result

Here we provide you some code to display the maze and the path carried out by the robot (the green cell) to solve the instance of the problem. Walls are represented as black cells, and garbage with brown cells.

In [103]:
def render_maze(grid, maze, garbage):
    height, width = len(maze), len(maze[0])
    # Render maze
    for i in range(width):
        for j in range(height):
            grid[j,i] = (200,200,200) if maze[j][i] in [0,2,3] else (0,0,0)

    # Render garbage in maze
    for g in garbage:
        grid[g[0],g[1]] = (139,69,19)
        
def find_agent(maze, width, height):
    for i in range(width):
        for j in range(height):
            if (maze[j][i] == 2):
                return (i,j)

def render_path(path, maze, garbage):
    height, width = len(maze), len(maze[0])
    solution_grid = BlockGrid(width, height, fill=(200, 200, 200))
    
    movementDict = {'DOWN':(0,1), 'UP':(0,-1), 'LEFT':(-1,0), 'RIGHT':(1,0)}
    agentPos = find_agent(maze, width, height)
    garbageRender = copy.deepcopy(garbage)
    
    # Initial position rendering
    render_maze(solution_grid, maze, garbageRender)
    solution_grid[agentPos[1],agentPos[0]] = (0, 255, 0)
    solution_grid.show()
    clear_output(wait=True)
    time.sleep(0.1)
    
    for action in path:
        # Update agent position
        agentPos = (agentPos[0] + movementDict[action.move][0], agentPos[1] + movementDict[action.move][1])
        
        # Update garbage list
        if ((agentPos[1], agentPos[0]) in garbageRender):
            garbageRender.remove((agentPos[1], agentPos[0]))
        
        # Render maze
        render_maze(solution_grid, maze, garbageRender)
        
        # Render agent, and update its position and the garbage list
        solution_grid[agentPos[1],agentPos[0]] = (0,255,0)
        
        solution_grid.show()
        clear_output(wait=True)
        time.sleep(0.1)

In [104]:
render_path(path_sol, problem_instance.maze, problem_instance.garbage)

,,,
,,,
,,,
,,,


You can easily try different instances of the problem just by changing the parameters when you call both the `main(...)` function and the `render_path(...)` function:

In [105]:
# path_sol, problem_instance = main(15, 15, 2021, 15, 6, 'RandomSearch')

In [106]:
# render_path(path_sol, problem_instance.maze, problem_instance.garbage)

## Experimental results

Once the algorithms have been implemented, you must study their performance. In order to do that, you must compare the quality of the solutions obtained, as well as the number of expanded nodes for instances of different maze sizes, number of walls and number of garbage cells.

Please, use new cells to insert code to carry out the experimental results and study of the algorithms.